In [13]:
!pip install python-igraph --user

In [14]:
__author__ = 'Kristy'
import time
import csv
from os import listdir
from os.path import join
from igraph import  *
import scipy as sc
import networkx as nx
import csv
import time
from scipy import stats
import numpy as np
import sys,igraph
import scipy.spatial.distance
from math import sqrt
import matplotlib as mpl
import matplotlib.pyplot as plt
from pylab import *

In [15]:
""" find the egonet of each node in the given graph. Return value will be a dictionary {vertex_index: [list of neighbours]}
    e.g.  {1:[0,2] , 2:[1,3]}
"""
def get_egonet (g):
    egonet = {k.index: g.neighbors(k,mode=ALL) for k in g.vs};
    return egonet;

""" To find the degree of nodes in the given graph, Return value will be a dictionary {vertex_index : degree}
    e.g. {1:2 , 2:2 }
"""
def get_di(g):
    neighbor_size = {k.index : g.degree(k,mode=ALL) for k in g.vs};
    return neighbor_size;

""" To find the clustering index of the nodes in the given graph. Return value will be a dictionary {vertex_index: clustering_index}
"""
def get_ci(g):
    clustering_index = {k.index : g.transitivity_local_undirected(vertices=k,mode=TRANSITIVITY_ZERO) for k in g.vs};
    return clustering_index;

""" To find the average number of two hop neighbors of all nodes in the given graph. Return value will be a dictionary {vertex_index: average_two_hop_neighbors}
"""
def get_dni(g):
    two_hop_neighbors = {};
    for key,value in get_egonet(g).items():
        avg_hop = mean([g.degree(k,mode=ALL) for k in value]);
        two_hop_neighbors[key] = avg_hop;
    return two_hop_neighbors;

""" To find the average clustering coefficient of all nodes in the given graph. Return value will be a dictionary {vertex_index: average_clustering coefficient}
"""
def get_cni(g):
    avg_ci = {}
    ci = get_ci(g)
    for key,value in get_egonet(g).items():
        temp = mean([ci[k] for k in value])
        avg_ci[key] = temp
    return avg_ci

""" To find the number of edges in the egonet of each node in the given graph. Return value will be a dictionary {vertex_index: edges_in_egonet}
"""
def get_eegoi(g):
    egonet = get_egonet(g);
    eegoi = {};
    for vertex in g.vs:
        sg = g.subgraph(egonet[vertex.index] + [vertex.index]);
        egonet_es = [(k.source,k.target) for k in sg.es]
        eegoi[vertex.index] = len(egonet_es);
    return eegoi;

""" To find the number of edges going out from the egonet of each node in the given graph. Return value will be a dictionary {vertex_index: outgoing_edges_from_egonet}
"""
def get_eoegoi(g):
    egonet = get_egonet(g);
    eoegoi = {};
    for vertex in g.vs:
        total_vs = [vertex.index];
        for k in egonet[vertex.index]:
            total_vs = total_vs + egonet[k] + [k];
        total_vs = list(set(total_vs));
        sg = g.subgraph(total_vs);
        total_es = [(k.source,k.target) for k in sg.es];
        sg_egonet = g.subgraph(egonet[vertex.index] + [vertex.index]);
        egonet_es = [(k.source,k.target) for k in sg_egonet.es];
        eoegoi[vertex.index] = len(list(set(total_es) - set(egonet_es)));
    return eoegoi;

""" To find the number of neighbors of the egonet of each node in the given graph. Return value will be a dictionary {vertex_index: neighbors_of_egonet}
"""
def get_negoi(g):
    egonet = get_egonet(g);
    negoi = {};
    for vertex in g.vs:
        egonet_vs = [vertex.index] + egonet[vertex.index];
        total_vs = [];
        for k in egonet[vertex.index]:
            total_vs = total_vs +egonet[k];
        total_vs = list(set(total_vs));
        total_vs = [i for i in total_vs if i not in egonet_vs];
        negoi[vertex.index] = len(total_vs);
    return negoi;

""" extract the features of each node in the given graph. Return value will be list of tuples of all features of each node
    e.g. if there are k nodes in graph then return value will be
    [(di0,di0,dni0,cni0,eego0,eoego0,negoi0),(di1,di1,dni1,cni1,eego1,eoego1,negoi1) ... (dik-1,dik-1,dnik-1,cnik-1,eegok-1,eoegok-1,negoik-1)]
"""
def get_features(g):
    di= get_di(g);
    ci= get_ci(g);
    dni= get_dni(g);
    cni=get_cni(g);
    eego=get_eegoi(g);
    eoego=get_eoegoi(g);
    negoi=get_negoi(g);
    all_features = [(di[v.index],ci[v.index],dni[v.index],cni[v.index],eego[v.index],eoego[v.index],negoi[v.index]) for v in g.vs];

    return all_features;

""" Get the signature vector of the graph. Return value will be a list of 35 values
    e.g [mn(f0),md(f0),std_dev(f0),skw(f0),krt(f0), ... mn(f6),md(f6),std_dev(f6),skw(f6),krt(f6)]
"""
def get_signature(g):
    all_features = get_features(g)
    num_nodes = len(all_features);
    signature = [];
    for k in range(0,7):
        feat_agg = [all_features[i][k] for i in range(0,num_nodes)];
        mn = mean(feat_agg);
        md = median(feat_agg);
        std_dev = np.std(feat_agg);
        skw = stats.skew(feat_agg);
        krt = stats.kurtosis(feat_agg);
        signature = signature + [mn,md,std_dev,skw,krt];
    del all_features;
    return signature;

""" find canberra distance between two signature vectors """
def get_canberra_distance(sign1,sign2):
    return abs(scipy.spatial.distance.canberra(sign1, sign2));

""" calculate threshold. two methods used. Method 1: median + 3 * range_mean, Method 2: mean + 3*sigma_c/sqrt(window size = 2)"""
def calculate_threshold(distances):
    n = 2
    moving_range = [abs(distances[i] - distances[i+1]) for i in range(0,len(distances)-1)];
    #range_mean = mean(moving_range);
    range_mean = sum(moving_range)/(len(moving_range)-1);
    med = median(distances)
    UCL = med + 3*range_mean
    """ threshold calculation method 2. uncomment the code below to find threshold by method2 """
    """
    dist_mean = mean(distances);
    sigma_c = range_mean / 1.128;
    UCL = dist_mean + (3 * (sigma_c/sqrt(n)));
    """
    return UCL;

""" determine anomalies on the basis of threshold"""
def anomalies(distances,u_threshold):
    anomalies = []
    for i in xrange(0, len(distances)-1):
        if distances[i] >= u_threshold and distances[i+1] >= u_threshold:
            anomalies.append(i+1);
    return anomalies;


def plot_dist(dists, u_threshold,filename):
    """
    Plot the (N-1) canberra distances comparing each graph with the previous
    """
    figure(num=None, figsize=(12, 6), dpi=80, facecolor='w', edgecolor='k')
    plt.plot(dists, "-o")
    axhline(y=u_threshold, ls='-', c='r',
            label='Threshold: $median + 3 * range mean = %0.2f$'%(u_threshold),
            lw=2)
    plt.grid(True)
    plt.legend(loc='best')
    plt.title("Anomaly Detection: ")
    plt.xlabel("Time Series Graphs")
    plt.ylabel("Canberra Distance")
    savefig(join('graph', filename+".png"),bbox_inches='tight')


""" This functions gives the dictionary of the text file as key and the graph object formed by igraph from the text file
    E.g. {'248_autonomous.txt': <igraph.Graph object at 0x000000001230B7D6D8>, '251_autonomous.txt':
    <igraph.Graph object at 0x000000001277D7C8>}
"""
def get_graphs(dir_path):
    file_paths = {f: join(dir_path,f) for f in listdir(dir_path)}
    graphs = {}
    for file,path in file_paths.items():
        try:
            fi = open(path,'r')
            v, e = fi.next().split()
            e_list = [(int(line.split()[0]),int(line.split()[1])) for line in list(fi)]
            g = Graph()
            g.add_vertices(int(v))
            g.add_edges(e_list)
            graphs[file] = g
        finally:
            fi.close()
    return graphs


#fetches dataset from graph kernel website
def return_dataset(file_name):
    #i = 'G_nci1'
    dd = datasets.fetch_dataset(file_name,verbose = True)
    graph_list = []
    node_attr = []
    for gg in dd.data:
        v = set([i[0] for i in gg[0]]).union(set([i[1] for i in gg[0]]))
        g_ = igraph.Graph()
        g_.add_vertices([str(i) for i in v])
        g_.add_edges([(str(i[0]), str(i[1])) for i in gg[0]])
        g_.simplify()
        graph_list.append(g_)
        g_.vs['idx'] = [str(i) for i in g_.vs.indices]
        node_attr.append(gg[1])
    data_y = dd.target
    return graph_list, data_y, node_attr
    

In [16]:
def apply_RF(feature_matrix, labels):
    model = RandomForestClassifier(n_estimators=500)
    res = cross_val_score(model, feature_matrix, labels, cv=10, scoring='accuracy')
    return np.mean(res)

In [11]:
"""
    run the program like this: python Netsimile.py C:/Users/Kristy/PycharmProjects/Netsimile/input_files
"""

if __name__ == "__main__":
    data = ["MUTAG","PTC_MR","PROTEINS_full","NCI1","NCI109","DD","COLLAB","REDDIT-BINARY","REDDIT-MULTI-5K","IMDB-BINARY","IMDB-MULTI"]
    data = ["REDDIT-MULTI-12K"]
    file = open("NETSMILE_res.csv",'a',newline = '')
    res_writer = csv.writer(file, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)
    for file_name in data:
        graph_list, data_y, data_nodes  = return_dataset(file_name)
        graph_signatures = [get_signature(k) for k in graph_list]
#         np.savetxt(file_name+"_NETSIMILE.txt", np.array(graph_signatures),  delimiter=",")
        acc = apply_RF(graph_signatures,data_y)
        print("accuracy:",acc)
        to_write = [file_name,acc]
        print(to_write)
        res_writer.writerow(to_write)
        file.flush()
    file.close()     
        

NameError: name 'datasets' is not defined

In [6]:
file = open("algos_time_res.csv",'a',newline = '')
res_writer = csv.writer(file, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)
data = ['ER1k', 'ER10k','facebook_g']
for d in data:
    graph = igraph.read('./'+d+'.gxl',format="graphml")
    print("graph successfuly loaded with {} number of nodes and {} number of edges".format(graph.number_of_nodes(), graph.number_of_edges()))

    start = time.time()
    sig = get_signature(graph)
    end = time.time()
    print("total time:", end-start)
    to_write = ['NetSmile',d,end-start]
    res_writer.writerow(to_write)
    file.flush()
    
file.close()

total time: 0.8296976089477539


C:\Appl\Anaconda\lib\site-packages\numpy\core\fromnumeric.py:3419: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
C:\Appl\Anaconda\lib\site-packages\numpy\core\_methods.py:188: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


total time: 11.22480034828186
total time: 215.12991333007812
total time: 3341.8454020023346


In [21]:
file = open("algos_time_res.csv",'a',newline = '')
res_writer = csv.writer(file, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)
data = ['ER1k', 'ER10k','facebook_g']
for d in data:
    graph = igraph.read('./'+d+'.gxl',format="graphml")
    graph2 = nx.read_graphml('./'+d+'.gxl')
    print("graph successfuly loaded with {} number of nodes and {} number of edges".format(graph2.number_of_nodes(), graph2.number_of_edges()))

    start = time.time()
    sig = get_signature(graph)
    end = time.time()
    print("total time:", end-start)
    to_write = ['NetSmile',d,end-start]
    res_writer.writerow(to_write)
    file.flush()
    
file.close()

graph successfuly loaded with 1000 number of nodes and 4912 number of edges
total time: 0.3381931781768799
graph successfuly loaded with 10000 number of nodes and 49762 number of edges
total time: 3.9021222591400146
graph successfuly loaded with 4039 number of nodes and 88234 number of edges
total time: 82.93905067443848


In [ ]:
file = open("algos_time_res.csv",'a',newline = '')
res_writer = csv.writer(file, delimiter=' ',quotechar='|', quoting=csv.QUOTE_MINIMAL)
data = ['twitter_g','github_g']
for d in data:
    graph = igraph.read('../graphs/'+d+'.gxl',format="graphml")
    
    print("done")
    start = time.time()
    sig = get_signature(graph)
    end = time.time()
    print("total time:", end-start)
    to_write = ['NetSmile',d,end-start]
    res_writer.writerow(to_write)
    file.flush()
    
file.close()

In [4]:
graph = igraph.Graph.Tree(8,2)
print(graph.vcount(), graph.ecount())

8 7


In [5]:
sig = get_signature(graph)
print(sig)

[1.75, 1.5, 0.82915619758885, 0.49338220021815865, -1.371900826446281, 0.0, 0.0, 0.0, 0.0, -3.0, 2.375, 2.5, 0.6548430685625714, -0.2735766875628648, -1.5867494959760038, 0.0, 0.0, 0.0, 0.0, -3.0, 1.75, 1.5, 0.82915619758885, 0.49338220021815865, -1.371900826446281, 2.375, 2.0, 0.8569568250501305, 0.3910423176326535, -0.4662743322770484, 2.0, 2.0, 0.8660254037844386, 1.1547005383792515, 1.0]


In [7]:
len(sig)

35